In [9]:
from flask import Flask, render_template, redirect, jsonify

import requests
import json
import pandas as pd

import pymongo

import nba_api
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import shotchartdetail, commonplayerinfo, leaguedashplayerstats, playerdashboardbygeneralsplits

from resources import mongo_user, mongo_pass, mongo_clusteruri

In [2]:
#mongo db initialization/test
conn = (f'mongodb+srv://{mongo_user}:{mongo_pass}@{mongo_clusteruri}')
client = pymongo.MongoClient(conn)

db = client.nba_db
collection = db.nba_reddit.find()

In [35]:
source = db.active_players.find()
destination = db.active_players_copy

In [36]:
for document in source:
    destination.replace_one(document, document, True)

In [6]:
player = 2738

# allshots = shotchartdetail.ShotChartDetail(player_id=player, season_nullable='2018-19').get_dict()

playerinfo = commonplayerinfo.CommonPlayerInfo(player_id=player).get_dict()

In [8]:
playerinfo['resultSets']

[{'name': 'CommonPlayerInfo',
  'headers': ['PERSON_ID',
   'FIRST_NAME',
   'LAST_NAME',
   'DISPLAY_FIRST_LAST',
   'DISPLAY_LAST_COMMA_FIRST',
   'DISPLAY_FI_LAST',
   'BIRTHDATE',
   'SCHOOL',
   'COUNTRY',
   'LAST_AFFILIATION',
   'HEIGHT',
   'WEIGHT',
   'SEASON_EXP',
   'JERSEY',
   'POSITION',
   'ROSTERSTATUS',
   'TEAM_ID',
   'TEAM_NAME',
   'TEAM_ABBREVIATION',
   'TEAM_CODE',
   'TEAM_CITY',
   'PLAYERCODE',
   'FROM_YEAR',
   'TO_YEAR',
   'DLEAGUE_FLAG',
   'NBA_FLAG',
   'GAMES_PLAYED_FLAG',
   'DRAFT_YEAR',
   'DRAFT_ROUND',
   'DRAFT_NUMBER'],
  'rowSet': [[2738,
    'Andre',
    'Iguodala',
    'Andre Iguodala',
    'Iguodala, Andre',
    'A. Iguodala',
    '1984-01-28T00:00:00',
    'Arizona',
    'USA',
    'Arizona/USA',
    '6-6',
    '215',
    15,
    '9',
    'Guard-Forward',
    'Active',
    1610612763,
    'Grizzlies',
    'MEM',
    'grizzlies',
    'Memphis',
    'andre_iguodala',
    2004,
    2019,
    'N',
    'Y',
    'Y',
    '2004',
    '1',
    '

In [27]:
playerdict = commonallplayers.CommonAllPlayers(season=2018, is_only_current_season=1).get_dict()
playerdict

{'resource': 'commonallplayers',
 'parameters': {'LeagueID': '00', 'Season': '2018', 'IsOnlyCurrentSeason': 1},
 'resultSets': [{'name': 'CommonAllPlayers',
   'headers': ['PERSON_ID',
    'DISPLAY_LAST_COMMA_FIRST',
    'DISPLAY_FIRST_LAST',
    'ROSTERSTATUS',
    'FROM_YEAR',
    'TO_YEAR',
    'PLAYERCODE',
    'TEAM_ID',
    'TEAM_CITY',
    'TEAM_NAME',
    'TEAM_ABBREVIATION',
    'TEAM_CODE',
    'GAMES_PLAYED_FLAG',
    'OTHERLEAGUE_EXPERIENCE_CH'],
   'rowSet': [[203518,
     'Abrines, Alex',
     'Alex Abrines',
     0,
     '2016',
     '2018',
     'alex_abrines',
     0,
     '',
     '',
     '',
     '',
     'Y',
     '00'],
    [203112,
     'Acy, Quincy',
     'Quincy Acy',
     0,
     '2012',
     '2018',
     'quincy_acy',
     0,
     '',
     '',
     '',
     '',
     'Y',
     '11'],
    [1629121,
     'Adams, Jaylen',
     'Jaylen Adams',
     1,
     '2018',
     '2019',
     'jaylen_adams',
     1610612737,
     'Atlanta',
     'Hawks',
     'ATL',
     'ha

In [37]:
playerdata = playerdict['resultSets'][0]['rowSet']
playerdata

[[203518,
  'Abrines, Alex',
  'Alex Abrines',
  0,
  '2016',
  '2018',
  'alex_abrines',
  0,
  '',
  '',
  '',
  '',
  'Y',
  '00'],
 [203112,
  'Acy, Quincy',
  'Quincy Acy',
  0,
  '2012',
  '2018',
  'quincy_acy',
  0,
  '',
  '',
  '',
  '',
  'Y',
  '11'],
 [1629121,
  'Adams, Jaylen',
  'Jaylen Adams',
  1,
  '2018',
  '2019',
  'jaylen_adams',
  1610612737,
  'Atlanta',
  'Hawks',
  'ATL',
  'hawks',
  'Y',
  '11'],
 [203500,
  'Adams, Steven',
  'Steven Adams',
  1,
  '2013',
  '2019',
  'steven_adams',
  1610612760,
  'Oklahoma City',
  'Thunder',
  'OKC',
  'thunder',
  'Y',
  '00'],
 [1628389,
  'Adebayo, Bam',
  'Bam Adebayo',
  1,
  '2017',
  '2019',
  'bam_adebayo',
  1610612748,
  'Miami',
  'Heat',
  'MIA',
  'heat',
  'Y',
  '00'],
 [1629061,
  'Adel, Deng',
  'Deng Adel',
  1,
  '2018',
  '2018',
  'deng_adel',
  1610612739,
  'Cleveland',
  'Cavaliers',
  'CLE',
  'cavaliers',
  'Y',
  '11'],
 [1629152,
  'Akoon-Purcell, DeVaughn',
  'DeVaughn Akoon-Purcell',
  0,


In [39]:
for player in playerdata:
    document = {
        'id' : player[0],
        'name_last_first' : player[1],
        'full_name' : player[2],
        'roster_status' : player[3],
        'from_year' : player[4],
        'to_year' : player[5],
        'player_code' : player[6],
        'team_id' : player[7],
        'team_city' : player[8],
        'team_name' : player[9],
        'team_abbreviation' : player[10],
        'team_code' : player[11],
        'games_played_bool' : player[12],
        'other_league_exp' : player[13]
    }
    
    db.active_players_copy.update({'id':document['id']}, document, True)

/Users/andrew/anaconda3/envs/bootcamp/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


In [42]:
db.active_players_copy.delete_many({"team_id": 0})


In [43]:
db.active_players_copy.count_documents({"team_id": 0})

0

In [45]:
for player in db.active_players_copy.find():
    print(player)

{'_id': ObjectId('5da78671c40d54eedfe4a33c'), 'id': 1629121, 'name_last_first': 'Adams, Jaylen', 'full_name': 'Jaylen Adams', 'roster_status': 1, 'from_year': '2018', 'to_year': '2019', 'player_code': 'jaylen_adams', 'team_id': 1610612737, 'team_city': 'Atlanta', 'team_name': 'Hawks', 'team_abbreviation': 'ATL', 'team_code': 'hawks', 'games_played_bool': 'Y', 'other_league_exp': '11'}
{'_id': ObjectId('5da78671c40d54eedfe4a348'), 'id': 203500, 'name_last_first': 'Adams, Steven', 'full_name': 'Steven Adams', 'roster_status': 1, 'from_year': '2013', 'to_year': '2019', 'player_code': 'steven_adams', 'team_id': 1610612760, 'team_city': 'Oklahoma City', 'team_name': 'Thunder', 'team_abbreviation': 'OKC', 'team_code': 'thunder', 'games_played_bool': 'Y', 'other_league_exp': '00'}
{'_id': ObjectId('5da78671c40d54eedfe4a353'), 'id': 1628389, 'name_last_first': 'Adebayo, Bam', 'full_name': 'Bam Adebayo', 'roster_status': 1, 'from_year': '2017', 'to_year': '2019', 'player_code': 'bam_adebayo', '

In [28]:
player = db.player_stats2.find({'player_id':1629121})

In [29]:
stats = []

for thing in player:
    print(thing)
    stats.append(thing)
    print(len(thing))

{'_id': ObjectId('5db125c334e8dd22c0e3f345'), 'player_id': 1629121, 'name': 'Jaylen Adams', 'team_id': 1610612737, 'team_abbr': 'ATL', 'age': 23.0, 'gp': 34, 'w': 13, 'l': 21, 'w_pct': 0.382, 'min': 427.5883333333333, 'fgm': 38, 'fga': 110, 'fg_pct': 0.345, 'fg3m': 25, 'fg3a': 74, 'fg3_pct': 0.338, 'ftm': 7, 'fta': 9, 'ft_pct': 0.778, 'oreb': 11, 'dreb': 49, 'reb': 60, 'ast': 65, 'tov': 28, 'stl': 14, 'blk': 5, 'blka': 7, 'pf': 45, 'pfd': 13, 'pts': 108, 'plus_minus': -57, 'nba_fant_pts': 306.5, 'dd2': 0, 'td3': 0, 'gp_rank': 372, 'w_rank': 374, 'l_rank': 202, 'w_pct_rank': 399, 'min_rank': 375, 'fgm_rank': 406, 'fga_rank': 390, 'fg_pct_rank': 468, 'fg3m_rank': 286, 'fg3a_rank': 294, 'fg3_pct_rank': 238, 'ftm_rank': 438, 'fta_rank': 443, 'ft_pct_rank': 218, 'oreb_rank': 380, 'dreb_rank': 382, 'reb_rank': 387, 'ast_rank': 285, 'tov_rank': 340, 'stl_rank': 348, 'blk_rank': 369, 'blka_rank': 156, 'pf_rank': 154, 'pfd_rank': 420, 'pts_rank': 404, 'plus_minus_rank': 377, 'nba_fant_pts_rank'

In [26]:
stats

[{'_id': ObjectId('5db125c334e8dd22c0e3f345'),
  'player_id': 1629121,
  'name': 'Jaylen Adams',
  'team_id': 1610612737,
  'team_abbr': 'ATL',
  'age': 23.0,
  'gp': 34,
  'w': 13,
  'l': 21,
  'w_pct': 0.382,
  'min': 427.5883333333333,
  'fgm': 38,
  'fga': 110,
  'fg_pct': 0.345,
  'fg3m': 25,
  'fg3a': 74,
  'fg3_pct': 0.338,
  'ftm': 7,
  'fta': 9,
  'ft_pct': 0.778,
  'oreb': 11,
  'dreb': 49,
  'reb': 60,
  'ast': 65,
  'tov': 28,
  'stl': 14,
  'blk': 5,
  'blka': 7,
  'pf': 45,
  'pfd': 13,
  'pts': 108,
  'plus_minus': -57,
  'nba_fant_pts': 306.5,
  'dd2': 0,
  'td3': 0,
  'gp_rank': 372,
  'w_rank': 374,
  'l_rank': 202,
  'w_pct_rank': 399,
  'min_rank': 375,
  'fgm_rank': 406,
  'fga_rank': 390,
  'fg_pct_rank': 468,
  'fg3m_rank': 286,
  'fg3a_rank': 294,
  'fg3_pct_rank': 238,
  'ftm_rank': 438,
  'fta_rank': 443,
  'ft_pct_rank': 218,
  'oreb_rank': 380,
  'dreb_rank': 382,
  'reb_rank': 387,
  'ast_rank': 285,
  'tov_rank': 340,
  'stl_rank': 348,
  'blk_rank': 369,


In [6]:
test = leaguedashplayerstats.LeagueDashPlayerStats(season='2018-19', per_mode_detailed='Per36').get_dict()

In [7]:
test

{'resource': 'leaguedashplayerstats',
 'parameters': {'MeasureType': 'Base',
  'PerMode': 'Per36',
  'PlusMinus': 'N',
  'PaceAdjust': 'N',
  'Rank': 'N',
  'LeagueID': None,
  'Season': '2018-19',
  'SeasonType': 'Regular Season',
  'PORound': None,
  'Outcome': None,
  'Location': None,
  'Month': 0,
  'SeasonSegment': None,
  'DateFrom': None,
  'DateTo': None,
  'OpponentTeamID': 0,
  'VsConference': None,
  'VsDivision': None,
  'TeamID': None,
  'Conference': None,
  'Division': None,
  'GameSegment': None,
  'Period': 0,
  'ShotClockRange': None,
  'LastNGames': 0,
  'GameScope': None,
  'PlayerExperience': None,
  'PlayerPosition': None,
  'StarterBench': None,
  'DraftYear': None,
  'DraftPick': None,
  'College': None,
  'Country': None,
  'Height': None,
  'Weight': None,
  'TwoWay': None},
 'resultSets': [{'name': 'LeagueDashPlayerStats',
   'headers': ['PLAYER_ID',
    'PLAYER_NAME',
    'TEAM_ID',
    'TEAM_ABBREVIATION',
    'AGE',
    'GP',
    'W',
    'L',
    'W_PCT'

In [11]:
dashboard = playerdashboardbygeneralsplits.PlayerDashboardByGeneralSplits(
        player_id=203507, 
        season='2018-19', 
        per_mode_detailed='Per36'
        ).get_dict()

In [12]:
dashboard

{'resource': 'playerdashboardbygeneralsplits',
 'parameters': {'MeasureType': 'Base',
  'PerMode': 'Per36',
  'PlusMinus': 'N',
  'PaceAdjust': 'N',
  'Rank': 'N',
  'LeagueID': None,
  'Season': '2018-19',
  'SeasonType': 'Regular Season',
  'PORound': None,
  'PlayerID': 203507,
  'Outcome': None,
  'Location': None,
  'Month': 0,
  'SeasonSegment': None,
  'DateFrom': None,
  'DateTo': None,
  'OpponentTeamID': 0,
  'VsConference': None,
  'VsDivision': None,
  'GameSegment': None,
  'Period': 0,
  'ShotClockRange': None,
  'LastNGames': 0},
 'resultSets': [{'name': 'OverallPlayerDashboard',
   'headers': ['GROUP_SET',
    'GROUP_VALUE',
    'GP',
    'W',
    'L',
    'W_PCT',
    'MIN',
    'FGM',
    'FGA',
    'FG_PCT',
    'FG3M',
    'FG3A',
    'FG3_PCT',
    'FTM',
    'FTA',
    'FT_PCT',
    'OREB',
    'DREB',
    'REB',
    'AST',
    'TOV',
    'STL',
    'BLK',
    'BLKA',
    'PF',
    'PFD',
    'PTS',
    'PLUS_MINUS',
    'NBA_FANTASY_PTS',
    'DD2',
    'TD3',
  